## ETL of Census Data

The "Escape the Bay" project was a success,  but generated 11 separate CSV Files!

Since each analysis was performed separately,  it was difficult to be able to draw correlations between the datasets.

The purpose of this ETL Homework is create a database were all the data can be stored, and queries can be written from dataset to dataset.

Thus the Tasks will be:

1) Extract data from all 11 CSVs and import into Pandas

2) Transform
   A. Eliminate un-needed data and missing data
   B. Based on the dataset size, determine whether to join the data in Pandas,  or in SQL
   
3) Create the Schema for the Escape-The-Bay Database

4) Upload into a POSTGRESQL database

5) Check the database and write a few sample queries using SQLALchemy;  

6) Plot with MathPlotLib to see the new visualizations that were previously not possilbe

7) Document!
   
References:  The data comes from Vanessa Oakes, Emily Todd, Stefan Zobrist and Rebecca Mih

## Transform

In [2]:
import csv
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json

In [3]:
# Census Quick Facts CSV
out_df = pd.read_csv('Quick Facts- Outside CA.csv')
california_df = pd.read_csv('Quick Facts-California.csv')
marital_ca_df = pd.read_csv('California - Marital 3.csv')
marital_out_df = pd.read_csv('Out of State - Marital.csv')

# Census API call and CSV creation
base_url = "https://api.census.gov/data/2017/acs/acs1/profile"
csv_path = "SF_All_OUT.csv"
sf_out_df = pd.read_csv(csv_path)

# Census Advanced Fact Finder CSV on Migration
CA_counties = pd.read_csv("CA_counties.csv")
non_CA_counties = pd.read_csv("non_CA_counties.csv")

# Census Advanced Fact Finder CSV on Industry 
file1 = "EstablishmentData2014.csv" 
file2 = "EstablishmentData2015.csv"
file3 = "EstablishmentData2016.csv"

df_original_1 = pd.read_csv(file1)
df_original_2 = pd.read_csv(file2)
df_original_3 = pd.read_csv(file3)


In [4]:
#  First determine where people migrate to,  from San Francisco.  Look at those moving within CA

#Top 5 in-CA counties
ranked_sf_out_df = sf_out_df.sort_values(by='Total', ascending=False)
sf_to_ca = ranked_sf_out_df.loc[ranked_sf_out_df["State Name"] == "California",:]
sf_to_ca.head(5)

#Transorm the data -- remove the State FIPs code and re-order the columns

sf_to_ca = sf_to_ca.rename(columns={"Total": "# Migrated from SF County (2017)"})

sftoca_summary = sf_to_ca.iloc[:5,3:7]

sftoca_summary.to_csv('ca_destinations.csv')
sftoca_summary = sftoca_summary.set_index(["County Name"])
sftoca_summary.head()

,State Name,# Migrated from SF County (2017),Margin of Error (+/-)
County Name,,,
Alameda County,California,10791,1127
San Mateo County,California,8995,1054
Contra Costa County,California,4085,631
Los Angeles County,California,3726,547
Santa Clara County,California,3383,447


In [5]:
#  Top destinations moving outside of CA

#Top 5 non-CA counties
sf_not_ca = ranked_sf_out_df.loc[ranked_sf_out_df["State Name"] != "California",:]
sf_not_ca.head(5)

#Transorm the data -- remove the State FIPs code and re-order the columns

sf_not_ca = sf_not_ca.rename(columns={"Total": "# Migrated from SF County (2017)"})

sfnotca_summary = sf_not_ca.iloc[:5,3:7]
sfnotca_summary = sfnotca_summary.set_index(["County Name"])

sfnotca_summary.to_csv('non_ca_destinations.csv')
sfnotca_summary.head()

,State Name,# Migrated from SF County (2017),Margin of Error (+/-)
County Name,,,
New York County,New York,1419,657
King County,Washington,1293,336
Multnomah County,Oregon,1094,282
Kings County,New York,887,300
Cook County,Illinois,635,223


In [7]:
#  Extract the QuickFacts data in CA and outside CA
california_df = pd.read_csv('Quick Facts-California.csv')

column_titles = ['Fact',
                'San Francisco County, California',
                'Alameda County, California', 
                 'San Mateo County, California',
                'Contra Costa County, California', 
                 'Los Angeles County, California',
                'Santa Clara County, California']

california_df = california_df.reindex(columns = column_titles)
california_df



,Fact,"San Francisco County, California","Alameda County, California","San Mateo County, California","Contra Costa County, California","Los Angeles County, California","Santa Clara County, California"
0,"Population estimates, July 1, 2017, (V2017)","884,363","1,663,190","771,410","1,147,439","10,163,507","1,938,153"
1,"Population estimates base, April 1, 2010, (V2...","805,193","1,510,261","718,500","1,049,200","9,818,696","1,781,671"
2,"Population, percent change - April 1, 2010 (es...",9.80%,10.10%,7.40%,9.40%,3.50%,8.80%
3,"Population, Census, April 1, 2010","805,235","1,510,271","718,451","1,049,025","9,818,605","1,781,642"
4,"Persons under 5 years, percent",4.50%,5.90%,5.70%,5.70%,6.10%,6.10%
5,"Persons under 18 years, percent",13.40%,20.70%,20.80%,22.80%,21.90%,22.20%
6,"Persons 65 years and over, percent",15.40%,13.50%,15.80%,15.30%,13.20%,13.10%
7,"Female persons, percent",49.00%,50.80%,50.70%,51.10%,50.70%,49.50%
8,"White alone, percent",53.10%,50.20%,60.60%,65.90%,70.90%,53.80%
9,"Black or African American alone, percent",5.50%,11.30%,2.80%,9.50%,9.00%,2.80%


In [8]:
out_df = pd.read_csv('Quick Facts- Outside CA.csv')
out_df.drop(columns = ["Fact Note"], inplace=True) 

column_titles = ['Fact',
                'San Francisco County, California',
                'New York County (Manhattan Borough), New York', 
                 'King County, Washington',
                'Multnomah County, Oregon', 
                 'Kings County (Brooklyn Borough), New York',
                'Cook County, Illinois']

out_df.reindex(columns = column_titles)

,Fact,"San Francisco County, California","New York County (Manhattan Borough), New York","King County, Washington","Multnomah County, Oregon","Kings County (Brooklyn Borough), New York","Cook County, Illinois"
0,"Population estimates, July 1, 2017, (V2017)","884,363","1,664,727","2,188,649","807,555","2,648,771","5,211,263"
1,"Population estimates base, April 1, 2010, (V2...","805,193","1,586,184","1,931,281","735,169","2,504,706","5,195,075"
2,"Population, percent change - April 1, 2010 (es...",9.80%,5.00%,13.30%,9.80%,5.80%,0.30%
3,"Population, Census, April 1, 2010","805,235","1,585,873","1,931,249","735,334","2,504,700","5,194,675"
4,"Persons under 5 years, percent",4.50%,4.80%,5.90%,5.60%,7.30%,6.20%
5,"Persons under 18 years, percent",13.40%,14.40%,20.40%,19.10%,22.90%,22.00%
6,"Persons 65 years and over, percent",15.40%,16.00%,13.00%,13.00%,13.50%,14.30%
7,"Female persons, percent",49.00%,52.60%,49.90%,50.50%,52.60%,51.40%
8,"White alone, percent",53.10%,64.40%,68.00%,79.50%,49.20%,65.60%
9,"Black or African American alone, percent",5.50%,17.90%,6.80%,6.00%,34.30%,24.00%


In [9]:
#dictionary for in-CA counties

base_url = "https://api.census.gov/data/2017/acs/acs1/profile"

ca_cty_name = ["San Francisco","Alameda","San Mateo","Contra Costa","Los Angeles","Santa Clara"]
ca_st_fips = ["06","06","06","06","06","06"]
ca_cty_fips = ["075","001","081","013","037","085"]

in_ca_dict = {
    "County Name": ca_cty_name,
    "State_FIPS": ca_st_fips,
    "County_FIPS": ca_cty_fips
}

in_ca_df = pd.DataFrame(in_ca_dict)
print(in_ca_df)

#dictionary for non-CA counties

nonca_cty_name = ["San Francisco","New York County","King County","Multnomah County","Kings County","Cook County"]
nonca_st_fips = ["06","36","53","41","36","17"]
nonca_cty_fips = ["075","061","033","051","047","031"]

non_ca_dict = {
    "County Name": nonca_cty_name,
    "State_FIPS": nonca_st_fips,
    "County_FIPS": nonca_cty_fips
}

non_ca_df = pd.DataFrame(non_ca_dict)
print(non_ca_df)

     County Name State_FIPS County_FIPS
0  San Francisco         06         075
1        Alameda         06         001
2      San Mateo         06         081
3   Contra Costa         06         013
4    Los Angeles         06         037
5    Santa Clara         06         085
        County Name State_FIPS County_FIPS
0     San Francisco         06         075
1   New York County         36         061
2       King County         53         033
3  Multnomah County         41         051
4      Kings County         36         047
5       Cook County         17         031


In [10]:
#collect median home values by county and add to data frames

ca_med_home_val = []
med_home_var = "DP04_0089E"
    
for county_id, state_id in zip(ca_cty_fips, ca_st_fips):
    med_home_val = requests.get(f"{base_url}?get={med_home_var}&for=county:{county_id}&in=state:{state_id}").json()
    ca_med_home_val.append(int(med_home_val[1][0]))
    
in_ca_df["Med_Home_Value"] = ca_med_home_val
in_ca_df.to_csv('ca_home_value.csv')
print(in_ca_df)

non_ca_med_home_val = []
med_home_var = "DP04_0089E"

for county_id, state_id in zip(nonca_cty_fips, nonca_st_fips):
    med_home_val = requests.get(f"{base_url}?get={med_home_var}&for=county:{county_id}&in=state:{state_id}").json()
    non_ca_med_home_val.append(int(med_home_val[1][0]))
    
non_ca_df["Med_Home_Value"] = non_ca_med_home_val
non_ca_df.to_csv('nonca_home_value.csv')
print(non_ca_df)

     County Name State_FIPS County_FIPS  Med_Home_Value
0  San Francisco         06         075         1104100
1        Alameda         06         001          781000
2      San Mateo         06         081         1087100
3   Contra Costa         06         013          642300
4    Los Angeles         06         037          588700
5    Santa Clara         06         085          971300
        County Name State_FIPS County_FIPS  Med_Home_Value
0     San Francisco         06         075         1104100
1   New York County         36         061          976100
2       King County         53         033          563600
3  Multnomah County         41         051          393900
4      Kings County         36         047          701800
5       Cook County         17         031          247600


In [11]:
#follow the same process for median gross rents

ca_med_rent = []
med_rent_var = "DP04_0134E"
    
for county_id, state_id in zip(ca_cty_fips, ca_st_fips):
    med_rent = requests.get(f"{base_url}?get={med_rent_var}&for=county:{county_id}&in=state:{state_id}").json()
    ca_med_rent.append(int(med_rent[1][0]))
    
in_ca_df["Med_Rent"] = ca_med_rent

in_ca_df.to_csv('ca_rents.csv')
print(in_ca_df)

non_ca_med_rent = []
med_rent_var = "DP04_0134E"

for county_id, state_id in zip(nonca_cty_fips, nonca_st_fips):
    med_rent= requests.get(f"{base_url}?get={med_rent_var}&for=county:{county_id}&in=state:{state_id}").json()
    non_ca_med_rent.append(int(med_rent[1][0]))
    
non_ca_df["Med_Rent"] = non_ca_med_rent
non_ca_df.to_csv('nonca_rents.csv')
print(non_ca_df)

     County Name State_FIPS County_FIPS  Med_Home_Value  Med_Rent
0  San Francisco         06         075         1104100      1836
1        Alameda         06         001          781000      1745
2      San Mateo         06         081         1087100      2259
3   Contra Costa         06         013          642300      1766
4    Los Angeles         06         037          588700      1402
5    Santa Clara         06         085          971300      2229
        County Name State_FIPS County_FIPS  Med_Home_Value  Med_Rent
0     San Francisco         06         075         1104100      1836
1   New York County         36         061          976100      1601
2       King County         53         033          563600      1555
3  Multnomah County         41         051          393900      1199
4      Kings County         36         047          701800      1367
5       Cook County         17         031          247600      1082


In [12]:
#follow the same process for home owner rate

ca_own_rate = []
home_own_var = "DP04_0046PE"
    
for county_id, state_id in zip(ca_cty_fips, ca_st_fips):
    own_rate = requests.get(f"{base_url}?get={home_own_var}&for=county:{county_id}&in=state:{state_id}").json()
    ca_own_rate.append(float(own_rate[1][0]))
    
in_ca_df["Home Own Rate"] = ca_own_rate
in_ca_df.to_csv('ca_homeowner_rates.csv')
print(in_ca_df)

nonca_own_rate = []
home_own_var = "DP04_0046PE"
    
for county_id, state_id in zip(nonca_cty_fips, nonca_st_fips):
    own_rate = requests.get(f"{base_url}?get={home_own_var}&for=county:{county_id}&in=state:{state_id}").json()
    nonca_own_rate.append(float(own_rate[1][0]))
    
non_ca_df["Home Own Rate"] = nonca_own_rate
non_ca_df.to_csv('nonca_homeowner_rates.csv')
print(non_ca_df)

     County Name State_FIPS County_FIPS  Med_Home_Value  Med_Rent  \
0  San Francisco         06         075         1104100      1836   
1        Alameda         06         001          781000      1745   
2      San Mateo         06         081         1087100      2259   
3   Contra Costa         06         013          642300      1766   
4    Los Angeles         06         037          588700      1402   
5    Santa Clara         06         085          971300      2229   

   Home Own Rate  
0           36.5  
1           53.3  
2           59.1  
3           67.8  
4           45.6  
5           56.8  
        County Name State_FIPS County_FIPS  Med_Home_Value  Med_Rent  \
0     San Francisco         06         075         1104100      1836   
1   New York County         36         061          976100      1601   
2       King County         53         033          563600      1555   
3  Multnomah County         41         051          393900      1199   
4      Kings County    

##  Data CleanUp Steps
Reduce the data size and clean up the naming (for easier reference later on)

df.drop()- Drop columns all the columns which have no important data

df.dropna() - Drop rows with NaN
df.reset_index() - Reset the index because we had dropped out a few rows
df[:x] - Drop rows, only keep x rows
df.rename() - Rename the colums with shorter names so the plots look ok

In [ ]:
# Rename the columns to shorten the names for plotting, look at the DataFrame
CA_counties = CA_counties.rename(columns={"San Francisco County, California": "San Francisco",
                                 "Alameda County, California":"Alameda",
                                 "San Mateo County, California":"San Mateo",
                                 "Contra Costa County, California":"Contra Costa",
                                "Los Angeles County, California":"Los Angeles",
                                "Santa Clara County, California":"Santa Clara"
                                            })

non_CA_counties = non_CA_counties.rename(columns={"San Francisco County, California":"San Francisco",
                                 "New York County (Manhattan Borough), New York":"NY (Manhattan), NY",
                                 "King County, Washington":"King, WA",
                                "Multnomah County, Oregon":"Multnomah, OR",
                                "Kings County (Brooklyn Borough), New York":"Kings (Brooklyn), NY",
                                "Cook County, Illinois":"Cook, IL",
                                "Travis County, Texas":"Travis, TX"
                                })


# Move the Facts into the index to get it out of the way since we don't need to clean the numbers in that column
# Making a new DF ca_data,  so  you can always refer to ca_df to see the line number of the row
ca_data = CA_counties.set_index("Fact")
non_ca_data = non_CA_counties.set_index("Fact")
non_ca_data.head()

## 4.0 Transform to numerical values
Because the raw data in the csv is formatted with $, %, or ',' Pandas will read all data as objects into the DataFrame

Clean the entire table, the user can specify the specific data fact (row) they wish to use
df.replace - Replace the %, $, , in the data to blank
df.apply(pd.to_numeric()) -- now change the objects in each column into numerics, "apply" will apply to all cols
Use errors = 'coerce' to force to an number. If there are alphanumerics, they will become 'NaN's and you will lose the text. In that case use errors = 'ignore'


In [ ]:
# First remove all the $, % or ,  characters from the dataframe

cols = ca_data.columns
cols_nonca = non_ca_data.columns

ca_data[cols] = ca_data[cols].replace({'\$': '', ',': '', '\%':'', '\"': ''}, regex=True)
non_ca_data[cols_nonca]=non_ca_data[cols_nonca].replace({'\$': '', ',': '', '\%':'', '\"': ''}, regex=True)


# convert all objects to numerics
# reference:  https://stackoverflow.com/questions/36814100/pandas-to-numeric-for-multiple-columns

clean_ca = ca_data.apply(pd.to_numeric, errors='coerce')
clean_ca.head()


non_ca_data = non_ca_data[cols_nonca].apply(pd.to_numeric, errors='coerce')
non_ca_data.head()

## Income and Mortgage data from census FactFinder Advanced Search utility
Reference:
Use the US Census FactFinder - Advanced Search functionality to get detailed in the area of Employment (including income), Housing (including Mortgage information), and Population demographics.

The utility is fairly easy to use -- but Warning - there is a LOT of data, and often times the data is repeated

Recommendations: Use the filtering and editing functions on the Advanced Search, BEFORE creating your CSV file.

Select all counties of interest first as a filter.
Use the graphical interface to input as many locations as you want (by city, county, state, etc)
Once you have selected all the key locations, you can save the query, which saves time if you are going to do other analyses later on

Edit out to the minimal data you need.

The census provides the data with calculated error, or percent error. Those can be filtered
Remove any columns of data you don't need. It's difficult to change column names with very large datasets, so better to minimize the number of columns if possible

The Income and Mortgage CSV in this file was filtered and edited on the Census website, with some additional description cleaning in Excel.

In [ ]:
# Files to load
ACS_data = pd.read_csv("2017_income_mortgage.csv")

ACS_data.head()